## Figure 7: Bias correction impact

In [ ]:
# spatial libraries
from shapely.geometry import Polygon
import rioxarray as rioxr
import geopandas as gpd
import xarray as xr
import xesmf as xe
import regionmask

# bias correction
from xclim import sdba

import pandas as pd
import numpy as np
import cftime
import os

# plotting libraries
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

# colors
cl = px.colors.qualitative.D3

In [ ]:
# Glacier dataset
glacier = gpd.read_file("/home/rooda/Dropbox/Patagonia/GIS South/Glaciers/RGI6_v2.shp")
glacier = glacier[glacier.Name == "Jorge Montt"] # select the glacier
glacier

In [ ]:
gcm       = "MIROC6"
ssp       = "ssp245"
climate   = "PMET"
bias_correction = ["MVA", "DQM", "MBC"]

baseline_period = slice("1980-01-01", "2014-12-31") # ISIMIP3b bias adjustment protocol
future_period   = baseline_period # Future period to bias correct

## Process climate projection for selected glacier

In [ ]:
# historical climate
pp_baseline  = xr.open_dataset("/home/rooda/OGGM_results/" + climate + "_OGGM_1980_2019m.nc")["prcp"]
t2m_baseline = xr.open_dataset("/home/rooda/OGGM_results/" + climate + "_OGGM_1980_2019m.nc")["temp"]
pp_baseline  = pp_baseline.sel(lat  = glacier.CenLat.iloc[0], lon = glacier.CenLon.iloc[0], method='nearest')
t2m_baseline = t2m_baseline.sel(lat = glacier.CenLat.iloc[0], lon = glacier.CenLon.iloc[0], method='nearest')
pp_baseline  = pp_baseline.sel( time  = baseline_period)
t2m_baseline = t2m_baseline.sel(time = baseline_period)

In [ ]:
os.chdir("/home/rooda/OGGM_results/Future_climate/")

pp_model_ssp = xr.open_dataset("PP_" + gcm + "_" + ssp + ".nc")["pr"]
pp_model_ssp = pp_model_ssp.interp(lat = pp_baseline.lat, lon = pp_baseline.lon)
t2m_model_ssp = xr.open_mfdataset("T2M_" + gcm + "_" + ssp + ".nc")["tas"]
t2m_model_ssp = t2m_model_ssp.interp(lat = t2m_baseline.lat, lon = t2m_baseline.lon)

In [ ]:
# 1. Scaling method ---------------------------------------------------------------------
qdm_t2m = sdba.adjustment.Scaling.train(ref = t2m_baseline, hist = t2m_model_ssp.sel(time = baseline_period), kind = "+", group = "time.month")
t2m_bc_mva = qdm_t2m.adjust(t2m_model_ssp.sel(time = future_period), interp="nearest").load()

qdm_pp  = sdba.adjustment.Scaling.train(ref = pp_baseline, hist = pp_model_ssp.sel(time  = baseline_period), kind = "*", group = "time.month")
pp_bc_mva  = qdm_pp.adjust(pp_model_ssp.sel(time  = future_period), interp="nearest").load()

# 2. Quantile Delta Mapping method --------------------------------------------------------------
qdm_t2m = sdba.adjustment.QuantileDeltaMapping.train(ref = t2m_baseline, hist = t2m_model_ssp.sel(time = baseline_period), kind = "+", group="time.month")
t2m_bc_dqm = qdm_t2m.adjust(t2m_model_ssp.sel(time = future_period), interp="nearest", extrapolation="constant").load()

qdm_pp  = sdba.adjustment.QuantileDeltaMapping.train(ref = pp_baseline, hist = pp_model_ssp.sel(time  = baseline_period), kind = "*", group="time.month")
pp_bc_dqm  = qdm_pp.adjust(pp_model_ssp.sel(time  = future_period), interp="nearest", extrapolation="constant").load()

# 3. Npdf Transform method  ------------------------------------------------------
qdm_t2m = sdba.QuantileDeltaMapping.train(t2m_baseline, t2m_model_ssp.sel(time = baseline_period), nquantiles = 20, kind = "+", group = "time.month")
scen_hist_t2m = qdm_t2m.adjust(t2m_model_ssp.sel(time = baseline_period))
scen_ssp_t2m  = qdm_t2m.adjust(t2m_model_ssp.sel(time = future_period))

qdm_pp = sdba.QuantileDeltaMapping.train(pp_baseline, pp_model_ssp.sel(time = baseline_period), nquantiles = 20, kind = "*", group = "time.month")
scen_hist_pp = qdm_pp.adjust(pp_model_ssp.sel(time = baseline_period))
scen_ssp_pp  = qdm_pp.adjust(pp_model_ssp.sel(time = future_period))

dref      = xr.Dataset(dict(tas = t2m_baseline,  pr = pp_baseline))
scen_hist = xr.Dataset(dict(tas = scen_hist_t2m, pr = scen_hist_pp))
scen_ssp  = xr.Dataset(dict(tas = scen_ssp_t2m,  pr = scen_ssp_pp))
scen_hist["time"] = dref.time # correct date (15 -> 01)

ref   = sdba.processing.stack_variables(dref) # Stack the variables (tas and pr)
scenh = sdba.processing.stack_variables(scen_hist)
scens = sdba.processing.stack_variables(scen_ssp)

ref, _, _          = sdba.processing.standardize(ref) # Standardize
allsim, savg, sstd = sdba.processing.standardize(xr.concat((scenh, scens), "time"))

hist = allsim.sel(time = scenh.time)
sim  = allsim.sel(time = scens.time)

out = sdba.adjustment.NpdfTransform.adjust(ref, hist, sim, base=sdba.QuantileDeltaMapping, 
                                           base_kws={"nquantiles": 20, "group": "time.month"}, n_iter=20, n_escore=1000)  
model_ssp_bc = sdba.processing.reordering(out, scens, group="time.month")
model_ssp_bc = sdba.processing.unstack_variables(model_ssp_bc)

t2m_bc_mbc = model_ssp_bc.tas.load()
pp_bc_mbc  = model_ssp_bc.pr.clip(min = 0).load()

## Plot

In [ ]:
fig = make_subplots(rows=2, cols=4, shared_yaxes = True, horizontal_spacing = 0.02, vertical_spacing = 0.09,
                    subplot_titles = ["a) Baseline", "b) Mean and variance scaling (MVA)", "c) Delta quantile mapping (DQM)", "d) Multivariate bias correction (MBCn)", "e)  Annual solid precipitation (mm)", "f) Degree months (ºC)"], 
                    specs=[[{"type": "Histogram2dContour"}, {"type": "Histogram2dContour"}, {"type": "Histogram2dContour"}, {"type": "Histogram2dContour"}],
                           [{"type": "Histogram2dContour", "colspan" : 2},          None,   {"type": "Histogram2dContour", "colspan" : 2},         None]])

range_x = [-8,8]
range_y = [0,1200]
range_z = [0,3]

ybins = dict(start=range_y[0], size=100, end=range_y[1])
xbins = dict(start=range_x[0], size=1,   end=range_x[1])

# a) Historical ----------------------------------------------------------
fig.add_trace(go.Histogram2dContour(x = t2m_baseline, y = pp_baseline, colorscale = ["rgba(235,235,235,1)", cl[0]], histnorm = "percent", line=dict(width=0.5, color= "white"),
                                    zmin = range_z[0], zmax =range_z[1], xbins = xbins, ybins = ybins, colorbar=dict(title='Percent (%)', x = 1.01, thickness = 12, y = 0.514, len = 1.029)), row = 1, col = 1)
fig.update_xaxes(title= "T2M (ºC)", range = range_x, row = 1, col = 1)
fig.update_yaxes(title= "PP (mm)", range = range_y, row = 1, col = 1)

# b) MVA ----------------------------------------------------------
fig.add_trace(go.Histogram2dContour(x = t2m_bc_mva, y = pp_bc_mva, colorscale = ["rgba(235,235,235,1)", cl[1]], histnorm = "percent", line=dict(width=0.5, color= "white"),
                                    zmin = range_z[0], zmax =range_z[1], xbins = xbins, ybins = ybins, colorbar=dict(x = 1.02,  thickness = 10)), row = 1, col = 2)
fig.update_xaxes(title= "T2M (ºC)", range = range_x, row = 1, col = 2)
fig.update_yaxes(range = range_y, row = 1, col = 2)

# c) DQM ----------------------------------------------------------
fig.add_trace(go.Histogram2dContour(x = t2m_bc_dqm, y = pp_bc_dqm, colorscale = ["rgba(235,235,235,1)", cl[2]], histnorm = "percent", line=dict(width=0.5, color= "white"),
                                    zmin = range_z[0], zmax =range_z[1], xbins = xbins, ybins = ybins, colorbar=dict(x = 1.03, thickness = 10)), row = 1, col = 3)
fig.update_xaxes(title= "T2M (ºC)", range = range_x, row = 1, col = 3)
fig.update_yaxes(range = range_y, row = 1, col = 3)

# d) MBC ----------------------------------------------------------
fig.add_trace(go.Histogram2dContour(x = t2m_bc_mbc, y = pp_bc_mbc, colorscale = ["rgba(235,235,235,1)", cl[4]], histnorm = "percent", line=dict(width=0.5, color= "white"),
                                    zmin = range_z[0], zmax =range_z[1], xbins = xbins, ybins = ybins,  colorbar=dict(x = 1.04, thickness = 20)), row = 1, col = 4)
fig.update_xaxes(title= "T2M (ºC)", range = range_x, row = 1, col = 4)
fig.update_yaxes(range = range_y, row = 1, col = 4)

# lines for a,b,c,d
fig.add_trace(go.Scatter(x=[0, 0], y=[0, 1200], mode='lines', line = dict(color='rgba(0,0,0,0.7)', width=0.5, dash='dash'), showlegend = False), row = 1, col = 1)
fig.add_trace(go.Scatter(x=[0, 0], y=[0, 1200], mode='lines', line = dict(color='rgba(0,0,0,0.7)', width=0.5, dash='dash'), showlegend = False), row = 1, col = 2)
fig.add_trace(go.Scatter(x=[0, 0], y=[0, 1200], mode='lines', line = dict(color='rgba(0,0,0,0.7)', width=0.5, dash='dash'), showlegend = False), row = 1, col = 3)
fig.add_trace(go.Scatter(x=[0, 0], y=[0, 1200], mode='lines', line = dict(color='rgba(0,0,0,0.7)', width=0.5, dash='dash'), showlegend = False), row = 1, col = 4)
fig.add_trace(go.Scatter(x=[5, 5], y=[0, 1200], mode='lines', line = dict(color='rgba(255,255,255,0.7)', width=1, dash='dot'), showlegend = False), row = 1, col = 1)
fig.add_trace(go.Scatter(x=[5, 5], y=[0, 1200], mode='lines', line = dict(color='rgba(255,255,255,0.7)', width=1, dash='dot'), showlegend = False), row = 1, col = 2)
fig.add_trace(go.Scatter(x=[5, 5], y=[0, 1200], mode='lines', line = dict(color='rgba(255,255,255,0.7)', width=1, dash='dot'), showlegend = False), row = 1, col = 3)
fig.add_trace(go.Scatter(x=[5, 5], y=[0, 1200], mode='lines', line = dict(color='rgba(255,255,255,0.7)', width=1, dash='dot'), showlegend = False), row = 1, col = 4)
fig.add_trace(go.Scatter(x=[-5, -5], y=[0, 1200], mode='lines', line = dict(color='rgba(255,255,255,0.7)', width=1, dash='dot'), showlegend = False), row = 1, col = 1)
fig.add_trace(go.Scatter(x=[-5, -5], y=[0, 1200], mode='lines', line = dict(color='rgba(255,255,255,0.7)', width=1, dash='dot'), showlegend = False), row = 1, col = 2)
fig.add_trace(go.Scatter(x=[-5, -5], y=[0, 1200], mode='lines', line = dict(color='rgba(255,255,255,0.7)', width=1, dash='dot'), showlegend = False), row = 1, col = 3)
fig.add_trace(go.Scatter(x=[-5, -5], y=[0, 1200], mode='lines', line = dict(color='rgba(255,255,255,0.7)', width=1, dash='dot'), showlegend = False), row = 1, col = 4)
    
# e) Annual solid pp --------------------------------------------------
fig.add_trace(go.Violin(x = pp_baseline.where(t2m_baseline <= 0, 0).resample(time='YS').sum(),  name = "Baseline", width=0.7, meanline_visible=True, box_visible=True, line_color=cl[0], opacity=0.6, showlegend = False), row = 2, col = 1)
fig.add_trace(go.Violin(x = pp_bc_mva.where(t2m_bc_mva <= 0, 0).resample(time='YS').sum(),      name = "MVA", width=0.7, meanline_visible=True, box_visible=True, line_color=cl[1],opacity=0.6, showlegend = False), row = 2, col = 1)
fig.add_trace(go.Violin(x = pp_bc_dqm.where(t2m_bc_dqm <= 0, 0).resample(time='YS').sum(),      name = "DQM", width=0.7, meanline_visible=True, box_visible=True, line_color=cl[2],opacity=0.6, showlegend = False), row = 2, col = 1)
fig.add_trace(go.Violin(x = pp_bc_mbc.where(t2m_bc_mbc <= 0, 0).resample(time='YS').sum(),      name = "MBC", width=0.7, meanline_visible=True, box_visible=True, line_color=cl[4],opacity=0.6, showlegend = False), row = 2, col = 1)
fig.update_xaxes(range = [1000, 7000], row = 2, col = 1)

# f) Degress days (?) --------------------------------------------------
fig.add_trace(go.Violin(x = t2m_baseline.where(t2m_baseline >= 0, 0).resample(time='YS').sum(),  name = "Baseline", width=0.7, meanline_visible=True, box_visible=True, line_color=cl[0], opacity=0.6, showlegend = False), row = 2, col =3 )
fig.add_trace(go.Violin(x = t2m_bc_mva.where(t2m_bc_mva >= 0, 0).resample(time='YS').sum(),      name = "MVA", width=0.7, meanline_visible=True, box_visible=True, line_color=cl[1],opacity=0.6, showlegend = False), row = 2, col = 3)
fig.add_trace(go.Violin(x = t2m_bc_dqm.where(t2m_bc_dqm >= 0, 0).resample(time='YS').sum(),      name = "DQM", width=0.7, meanline_visible=True, box_visible=True, line_color=cl[2],opacity=0.6, showlegend = False), row = 2, col = 3)
fig.add_trace(go.Violin(x = t2m_bc_mbc.where(t2m_bc_mbc >= 0, 0).resample(time='YS').sum(),      name = "MBC", width=0.7, meanline_visible=True, box_visible=True,  line_color=cl[4],opacity=0.6, showlegend = False), row = 2, col = 3)
fig.update_xaxes(range = [0, 17], row = 2, col = 3)
    
# Layout for all ------------------------------------------------------------------------------
fig.update_yaxes(ticks="outside", griddash = "dot", gridcolor = "rgba(255,255,255,0.7)", title_standoff = 0, zeroline = False,  showline = True, linecolor = 'rgba(0,0,0,0.5)', linewidth = 1, mirror=True)
fig.update_xaxes(ticks="outside", griddash = "dot", gridcolor = "rgba(255,255,255,0.7)", title_standoff = 0, zeroline = False,  showline = True, linecolor = 'rgba(0,0,0,0.5)', linewidth = 1, mirror=True)

fig.update_layout(autosize = False, width = 1200, height = 800, margin = dict(l=5, r=5, b=5, t=30))
fig.update_layout(paper_bgcolor='white', plot_bgcolor='rgba(235,235,235,1)')

fig.write_image("/home/rooda/Dropbox/Patagonia/MS2 Results/Figure_7_BCM.png", scale=4)
fig.show()